In [1]:
import pandas as pd
import os

# 测试一下简单的例子
from torchtext import data 
from torchtext.vocab import Vectors
from torch.nn import init

In [2]:
import torch
print(torch.__version__)

0.4.1


In [3]:
tokenize = lambda x:x.split()

TEXT = data.Field(sequential=True,tokenize=tokenize,batch_first=True,fix_length=5)

In [4]:
# 自定义Dataset
class MyDataset(data.Dataset):
    def __init__(self,path,text_field,**kwargs):
        fields = [('text',text_field)]
        examples = []
        csv_data = pd.read_csv(path)
        for text in csv_data['text']:
            examples.append(data.Example.fromlist([text],fields))
        
        super(MyDataset,self).__init__(examples,fields,**kwargs)

In [5]:
train_path = './data/test_text.csv'
train = MyDataset(train_path,text_field =TEXT)

In [6]:
len(train)

4

In [7]:
train[0].__dict__.keys()

dict_keys(['text'])

In [8]:
train[0].text

['i', 'love', 'you']

In [9]:
# 通过预训练的词向量来构建词表
cache = '.vector_cache'
if not os.path.exists(cache):
    os.mkdir(cache)
vector_path = './data/test_vector.txt'
vectors = Vectors(name=vector_path,cache=cache)

In [10]:
vectors.unk_init = init.xavier_uniform_

In [11]:
vectors[0]

tensor([[ 0.2633, -0.7401,  1.1374]])

In [12]:
TEXT.build_vocab(train,train,train,min_freq=2,vectors=vectors)

In [13]:
TEXT.vocab.vectors.shape

torch.Size([8, 3])

In [14]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             'i': 2,
             'you': 3,
             'hate': 4,
             'him': 5,
             'love': 6,
             'but': 7})

In [15]:
len(TEXT.vocab)

8

In [16]:
for item in train:
    print(item.text)

['i', 'love', 'you']
['but', 'you', 'love', 'him']
['i', 'hate', 'him']
['i', 'hate', 'you']


In [17]:
train_iter = data.BucketIterator(dataset=train, batch_size=1, shuffle=False, sort_within_batch=False,
                                     repeat=False, device=-1)

In [18]:
for item in train_iter:
    print(item.text)

tensor([[2, 6, 3, 1, 1]])
tensor([[7, 3, 6, 5, 1]])
tensor([[2, 4, 5, 1, 1]])
tensor([[2, 4, 3, 1, 1]])


In [19]:
word2vec_dir = '../data/mini/word2vec_data/'
article_embedding_path = os.path.join(word2vec_dir,'article_{}.txt'.format(50))

In [20]:
with open(article_embedding_path,'r') as f:
    for line in f:
        print(len(line.strip().split()))
        break

51


In [21]:
cache = '.vector_cache'
if not os.path.exists(cache):
    os.mkdir(cache)

In [23]:
vectors = Vectors(name=article_embedding_path,cache=cache)

100%|██████████| 6052/6052 [00:00<00:00, 52334.33it/s]


In [29]:
vectors